In [1]:
import warnings; warnings.filterwarnings(action='ignore')
%matplotlib inline
#for Netcdf manipulation
import xarray as xr
from netCDF4 import Dataset
import netCDF4 as nc

#for array manipulation
import numpy as np
import pandas as pd
from pandas import DataFrame
from pandas import Grouper

In [2]:
# read in files
sic_file = '/Users/fridaperez/Developer/repos/volume-thickness/SIT_files/sseaice_conc_months5-10_02-18v03r1.nc'  # prepping path for xr
data = xr.open_dataset(sic_file) # opening sit for xarray
sic = data['seaice_conc_monthly_cdr']
ln=data['longitude']
lt=data['latitude']
time=data["time"]
#turn to arrays
latitude=lt.values
longitude=ln.values
conc=data['seaice_conc_monthly_cdr'].values
lat_new=data['longitude'].values
lon_new=data['latitude'].values

In [3]:
areamaskfile = '/Users/fridaperez/Desktop/pss25area_v3.dat'
with open(areamaskfile, 'rb') as famsk:
    mask2 = np.fromfile(famsk, dtype=np.uint32)
    areamaskf = np.array(mask2, dtype=float)/1000 # scaled by 1000 (divide by 1000 to get square km)

In [4]:
areamaskf

array([444.053, 444.983, 445.91 , ..., 462.098, 461.12 , 460.139])

In [5]:
mask = areamaskf.reshape(332,316)

In [6]:
# area = np.zeros((np.shape(conc)[0], np.shape(conc)[1], np.shape(conc)[2]))
#
#
# def multiplyDivideArray(arr1, arr2):
#     for i in range(np.shape(arr1)[0]): #time
#         for j in range(np.shape(arr1)[1]): #lat
#             for k in range(np.shape(arr1)[2]):  #long
#                 if arr1[i,j,k] >=  .15 and arr1[i,j,k] <= 1.0:
#                     arr2[i,j,k] = arr1[i,j,k]*625
#     return arr2
#
#
# area = multiplyDivideArray(conc,area)
# #print(area)

In [7]:
mask.shape

(332, 316)

In [8]:
conc.shape

(97, 332, 316)

In [9]:
h = conc*mask

In [10]:
conc[0]

array([[0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)

In [11]:
g = np.multiply(conc,mask)

In [12]:
g.shape

(97, 332, 316)

In [14]:
area_array = xr.DataArray(g)

In [15]:
da = xr.DataArray(
    data=area_array,
    dims=["time","ygrid","xgrid"],
    coords=dict(
        time=time,
        longitude=(["ygrid","xgrid"],longitude),
        latitude=(["ygrid","xgrid"],latitude),
    ),
    attrs=dict(
        description="Sea Ice Area",
        units="km2",
    ),
)

In [16]:
#da.to_netcdf('/Users/fridaperez/Developer/repos/local_repo/Area_SH_Filled-316x332.nc') # remember to change paths

In [ ]:
# file = '/Users/fridaperez/Developer/repos/volume-thickness/SIA_files/Area_CYS.nc' # again remember to change paths
# datos = xr.open_dataset(file)

In [ ]:
#newset=datos.rename_vars({'__xarray_dataarray_variable__':'area'})

In [ ]:
# df=newset['area']

In [ ]:
# m = Basemap(projection= 'spaeqd',boundinglat=-50,lon_0=-180,resolution='l')
# x, y = m(longitude,latitude)
# fig = plt.figure(figsize=(12,12))
# m.fillcontinents(color='white',lake_color='white')
# m.drawcoastlines()
# m.drawparallels(np.arange(-80.,0.,20.))
# m.drawmeridians(np.arange(-180.,181.,20.))
# m.drawmapboundary(fill_color='lightblue')
# m.contourf(x,y,df[40],40,cmap=cmocean.cm.thermal)
# transform=ccrs.SouthPolarStereo()
# plt.colorbar()